## Predicting Market Outcomes: Utilizes Data Research and DataFrames to Cluster Data

Utilized Traits:
* Betting Data (Investment Amounts, Frequency, Wallets)
* Outcome Data (Distribution, Winner)
* Market Details (Category, Date, Time Length)

Open DataFrames

In [99]:
import pandas as pd

marketOutcomes = pd.read_csv('data/silver/marketOutcomes.csv')
markets = pd.read_csv('data/silver/markets_with_ai_categories.csv')

Obtain Addresses

In [100]:
addresses = markets['marketMakerAddress']

In [101]:
import os
from dotenv import load_dotenv
import requests
import pprint
import pandas as pd
load_dotenv()
from datetime import datetime, timedelta
POLYGON_API_KEY = os.getenv("POLYGONSCAN_API_KEY")
print(POLYGON_API_KEY)
def time_decoder(address: str, iter: int = None, sz: str = None, hundredPlusBuys = None, lessThanHundredBuys = None): 
    dictForAddys = {}
    if sz is not None:
        if sz == 'large':
            dictForAddys = hundredPlusBuys[iter]
        elif sz == 'small':
            dictForAddys = lessThanHundredBuys[iter]
    
    else:
        try:
            dictForAddys = pd.read_csv(f'data/bronze/contract_buy_{address}.csv')
        except FileNotFoundError:
            return None
    try:
        
        people = [addy for addy in dictForAddys['buyer']]  
        people = [a.lower() for a in people]
        
    except KeyError:
        return None
    url = (
      f'https://api.polygonscan.com/api'
      f'?module=account'
      f'&action=tokentx'
      f'&address={address}'
      f'&startblock=0'
      f'&endblock=99999999'
      f'&apikey={POLYGON_API_KEY}'
    )
    response = requests.get(url)


    transactionDict = response.json()
    status = transactionDict.get("status")
    if status != '1':
        print(f"Error: {transactionDict.get('message')}")
        return None


    timeDict = {}
    for row in transactionDict.get('result', []):
      
      if row.get('from', '').lower() in people:
            try:
               timeDict[row.get('from','').lower()].append(row.get('timeStamp'))
            except KeyError:
               timeDict[row.get('from', '').lower()] = [row.get('timeStamp')]

    from datetime import datetime
    allTimes = [ts for ts_list in timeDict.values() for ts in ts_list]

    
    minTimestamp = min(datetime.fromtimestamp(int(x)) for x in allTimes)
    for buyer, lst in timeDict.items():
        lst_dt = sorted(datetime.fromtimestamp(int(x)) for x in lst)
        timeDict[buyer] = [
            int((ts - minTimestamp).total_seconds())   
            for ts in lst_dt
        ]
    
    buyers= dictForAddys['buyer'].str.lower()
    dictForAddys['timeStampSinceFirst'] = [timeDict[addy].pop(0) if addy in timeDict and timeDict[addy] else None for addy in buyers]
    dictForAddys.drop('timeStamp', axis=1, inplace=True)
    dictForAddys.to_csv(f'data/bronze/contract_official_buys_{address}.csv')

    return timeDict

JCBJI6AJS9E71ZVH58E8IT9E8JZBCCRQEE


In [ ]:
timerStamper = []
turn = 0
for index, market in markets.iterrows():
    turn += 1
    print(f"Turn {turn} of {len(markets)}")
    try:
        timeStamp  = time_decoder(market["marketMakerAddress"])
    except:
        continue    
    print("Yahoo News")
    try:
        buyScans = pd.read_csv(f'data/bronze/contract_official_buys_{market["marketMakerAddress"]}.csv')
    except FileNotFoundError:
        print(f"Error: 1")
        continue
    try:
        buyScans75Index = max(buyScans['timeStampSinceFirst'].tolist())*.75
        buyScans75 = buyScans[buyScans['timeStampSinceFirst'] < buyScans75Index]
        
    except:
        print(f"Error: Unicorn")
        continue
    try:
        filtered = marketOutcomes.loc[marketOutcomes["marketMakerAddress"] == market["marketMakerAddress"], "index"]
        marketOutcomeIndex = filtered.iloc[0]
    except:
        print(f"Error: 2")
        continue
    
    try:
        buysIndex0 = buyScans75[buyScans75['outcomeIndex'] == 0]['investmentAmount']
        buysIndex1 = buyScans75[buyScans75['outcomeIndex'] == 1]['investmentAmount']
    except:
        print(f"Error: 3")
        continue
    
    try:
        total = sum(buyScans75['investmentAmount'].tolist())
        buyScans75[buyScans75['investmentAmount'] > .05*total]
        buyScansVC = buyScans75['outcomeIndex'].value_counts()
        whale0 = buyScansVC.get(0, 0)
        whale1 = buyScansVC.get(1, 0)
    except:
        print("Error: 5")
        continue
    
    try: 
        sz = len(buyScans75)
        finalRatioVC = buyScans75['outcomeIndex'].value_counts()
        count0 = finalRatioVC.get(0, 0)
        count1 = finalRatioVC.get(1, 0)
        ratio = count0 / (count0 + count1)
    except:
        print("Error: 6")
        continue
        
    try:
        timerStamper.append({'Market': market['marketMakerAddress'], 'Category': market['category'], 'TimeStamps': timeStamp, 'BuysIndex0': buysIndex0.tolist(), 'BuysIndex1': buysIndex1.tolist(), 'OutcomeIndex': marketOutcomeIndex, 'Whale0': whale0, 'Whale1': whale1, 'FinalRatio': ratio})
        print("Added")
    except:
        print("Timer Failure")
        
    print("Success")
timerStamper = pd.DataFrame(timerStamper)
timerStamper.to_csv('data/silver/timerStamper1.csv', index=False)

Turn 1 of 1138
Yahoo News
Added
Success
Turn 2 of 1138
Yahoo News
Added
Success
Turn 3 of 1138
Yahoo News
Added
Success
Turn 4 of 1138
Yahoo News
Added
Success
Turn 5 of 1138
Yahoo News
Added
Success
Turn 6 of 1138
Yahoo News
Added
Success
Turn 7 of 1138
Yahoo News
Added
Success
Turn 8 of 1138
Yahoo News
Added
Success
Turn 9 of 1138
Yahoo News
Added
Success
Turn 10 of 1138
Yahoo News
Added
Success
Turn 11 of 1138
Yahoo News
Added
Success
Turn 12 of 1138
Yahoo News
Added
Success
Turn 13 of 1138
Yahoo News
Added
Success
Turn 14 of 1138
Yahoo News
Added
Success
Turn 15 of 1138
Yahoo News
Added
Success
Turn 16 of 1138
Yahoo News
Added
Success
Turn 17 of 1138
Yahoo News
Added
Success
Turn 18 of 1138
Yahoo News
Added
Success
Turn 19 of 1138
Turn 20 of 1138
Yahoo News
Added
Success
Turn 21 of 1138
Yahoo News
Added
Success
Turn 22 of 1138
Yahoo News
Added
Success
Turn 23 of 1138
Yahoo News
Added
Success
Turn 24 of 1138
Yahoo News
Added
Success
Turn 25 of 1138
Yahoo News
Added
Success
Turn 26 o

Creating Official ML Model Training Data

In [17]:
import pandas as pd
timerStamper = pd.read_csv("data/silver/timerStamper1.csv")

from sklearn.preprocessing import LabelEncoder
badIndeces = []
encoder = LabelEncoder()
timerStamper['CategoryEncoded'] = encoder.fit_transform(timerStamper['Category'])
from datetime import timedelta
import ast
dataSector = []
for iter, line in timerStamper.iterrows():
    buysIndex0 = ast.literal_eval(line['BuysIndex0'])
    buysIndex1 = ast.literal_eval(line['BuysIndex1'])
    try:
        timeDict = ast.literal_eval(line['TimeStamps'])
        allDeltas = []
        for deltas in timeDict.values():
            allDeltas.extend(deltas)
        maxDelta = max(allDeltas)
    except:
        badIndeces.append(iter)
        print("Error: 7")
        continue
    # Total length of time of the market
    if None in [line['CategoryEncoded'], maxDelta ,len(buysIndex0), sum(buysIndex0), len(buysIndex1), sum(buysIndex1), line['Whale0'], line['Whale1'], line['FinalRatio']] or (len(buysIndex0) + len(buysIndex1)) < 10:
        badIndeces.append(iter)
        continue
    dataSector.append([line['CategoryEncoded'], maxDelta ,len(buysIndex0), sum(buysIndex0), len(buysIndex1), sum(buysIndex1), line['Whale0'], line['Whale1'], line['FinalRatio'], line['OutcomeIndex']])
    print(line['OutcomeIndex'])
    
timerStamper.drop(badIndeces, inplace=True)
timerStamper.reset_index(drop=True, inplace=True)
timerStamper.to_csv('data/silver/timerStamper1.csv', index=False)

1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
1
1
1
0
1
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
0
0
0
0
1
1
0
1
0
1
0
0
1
0
1
0
0
0
0
1
0
0
1
0
0
0
1
1
0
1
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
0
0
1
1
0
0
1
1
0
0
1
0
0
1
1
0
0
1
0
1
0
0
1
0
0
0
1
0
0
0
1
1
1
0
0
1
0
0
0
0
0
1
1
1
0
1
1
0
1
0
0
0
0
0
1
1
0
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
0
1
0
0
1
1
0
1
0
1
1
0
0
1
0
1
1
1
0
0
1
1
0
0
1
0
0
1
0
0
0
0
1
0
1
1
1
1
1
0
1
1
1
0
0
1
1
0
0
1
1
1
0
0
1
1
0
0
0
0
1
0
0
1
1
0
1
0
0
1
1
0
0
0
0
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
0
1
0
1
1
0
0
0
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
0
1
1
1
1
1
0
0
1
0
0
0
1
1
1
1
0
0
1
1
0
1
0
1
1
0
1
0
1
0
0
0
0
1
0
0
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
1
1
0
1
0
0
1
1
1
0
0
1
0
0
0
0
0
1
0
0
1
1
1
1
0
1
1
1
0
1
0
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
0
1
0
1
0
1
1
0
1
0
0
1
1
0
0
0
1
1
1
1
1
1
0
0
0
0
0
1
0
1
1
1
0
1
0
1
1
0
1
1
1
0
1
0
1
1
1
1
1
0
0
1
0
1
1
1
0
1
1
1
0
0
1
0
1
1
1
0


Splitting Data into Test and Train

In [ ]:
import torch

import torch
from torch.utils.data import Dataset, DataLoader

class BinaryDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32) # Use float for binary labels
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


totalData = len(dataSector)

trainCap = totalData*4 // 5

trainDataInputs = dataSector[:trainCap]

testDataInputs = dataSector[trainCap:]

trainResults = [x[9] for x in trainDataInputs]

testResults = [x[9] for x in testDataInputs]

trainData = BinaryDataset(trainDataInputs, trainResults)
testData = BinaryDataset(testDataInputs, testResults)


batch_size = 10
shuffle = True
dataLoader = DataLoader(trainData, batch_size=batch_size, shuffle=shuffle)
dataLoaderTest = DataLoader(testData, batch_size=batch_size, shuffle=shuffle)




Creating the Model

Model Architecture taken from here : https://medium.com/data-science/pytorch-tabular-binary-classification-a0368da5bb89

Key Point: Relu serves to fid complex relationships between variables
Key Point: Normalizing to keep everything within 0 to 1 scope and not put too much weight on anything
Key Point: Linear Layers Attempt to make reason out of data

In [19]:
import torch
import torch.nn as nn
class MarketPredictor(nn.Module):
    "Initializes multi-class classification model"
    def __init__(self, input_features=9, output_features=1, hidden_units=16):
        super().__init__()
        self.layer_1 = nn.Linear(input_features, hidden_units) 
        self.layer_2 = nn.Linear(hidden_units, hidden_units)
        self.layer_out = nn.Linear(hidden_units, output_features) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(hidden_units)
        self.batchnorm2 = nn.BatchNorm1d(hidden_units)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

MarketPredictorModel = MarketPredictor(input_features=10,
                    output_features=1,
                    hidden_units=16)

lossFn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(MarketPredictorModel.parameters(), lr=0.001)



A way to Analyze Accuracy (Literally just a percent)

In [20]:
def binaryAccuracy(actualOutcomes, predProbs, threshold=0.5):
    preds = (predProbs > threshold).float()
    correct = (preds == actualOutcomes).float().sum()
    acc = correct / actualOutcomes.shape[0]
    return acc * 100

Training Data

Both training and testing

In [21]:
import random

'''
def train(epochs: int):
  rates = []
  maxAcc = 0
  for epoch in range(epochs):
    MarketPredictorModel.train()
    trainIndex = random.randint(736)
    binaryPredictions = MarketPredictorModel(trainData[trainIndex]).squeeze()
    loss = lossFn(binaryPredictions, trainResults)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    probs1 = torch.sigmoid(binaryPredictions)
    accuracy = binaryAccuracy(trainResults, probs1)
    MarketPredictorModel.eval()
    
    with torch.inference_mode():
      indexTest = random.randint(100)
      testPredictions = MarketPredictorModel(testData[indexTest]).squeeze()
      lossTest = lossFn(testPredictions, testResults[indexTest])
      probs = torch.sigmoid(testPredictions)
      accTest = binaryAccuracy(testResults, probs)
      
      
      print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {accuracy:.2f}% | Test Loss: {lossTest:.5f} | Test Acc: {accTest:.4f}")
      maxAcc = max(maxAcc, accTest)
      rates.append(accTest)
      
  import statistics
  return maxAcc, statistics.mean([r.item() for r in rates])
'''



def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.


    for i, data in enumerate(dataLoader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = MarketPredictorModel(inputs)
        outputs = outputs.squeeze()
        loss = lossFn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        outputsSig = torch.sigmoid(outputs)

        
def test_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Iterate over the test data and generate predictions
    for i, data in enumerate(dataLoaderTest):
        inputs, labels = data

        # Generate predictions
        outputs = MarketPredictorModel(inputs)
        outputs = outputs.squeeze()
        loss = lossFn(outputs, labels)

        # Gather data and report
        running_loss += loss.item()
        outputsSig = torch.sigmoid(outputs)
        accuracy = binaryAccuracy(labels, outputsSig)
        
        print(f"Epoch: {epoch_index} | Test Loss: {loss:.5f} | Test Acc: {accuracy:.4f}")
        
        

Training Data to be 85% accuracte

In [22]:
maxAcc = 0
mean = 0
epochs =200

for epoch in range(epochs):
    print(f"Epoch {epoch} of {epochs}")
    train_one_epoch(epoch, None)
    test_one_epoch(epoch, None)
    
torch.save(MarketPredictorModel.state_dict(), "MarketPredictor.pt")

Epoch 0 of 200


Epoch: 0 | Test Loss: 0.60456 | Test Acc: 70.8333
Epoch: 0 | Test Loss: 0.69069 | Test Acc: 79.1667
Epoch: 0 | Test Loss: 0.66730 | Test Acc: 58.3333
Epoch: 0 | Test Loss: 0.62188 | Test Acc: 79.1667
Epoch: 0 | Test Loss: 0.64207 | Test Acc: 75.0000
Epoch: 0 | Test Loss: 0.64822 | Test Acc: 58.3333
Epoch: 0 | Test Loss: 0.53936 | Test Acc: 83.3333
Epoch: 0 | Test Loss: 0.59176 | Test Acc: 81.2500
Epoch 1 of 200
Epoch: 1 | Test Loss: 0.68309 | Test Acc: 79.1667
Epoch: 1 | Test Loss: 0.60463 | Test Acc: 75.0000
Epoch: 1 | Test Loss: 0.58545 | Test Acc: 79.1667
Epoch: 1 | Test Loss: 0.65504 | Test Acc: 66.6667
Epoch: 1 | Test Loss: 0.55668 | Test Acc: 75.0000
Epoch: 1 | Test Loss: 0.50289 | Test Acc: 87.5000
Epoch: 1 | Test Loss: 0.62327 | Test Acc: 79.1667
Epoch: 1 | Test Loss: 0.59279 | Test Acc: 81.2500
Epoch 2 of 200
Epoch: 2 | Test Loss: 0.60584 | Test Acc: 83.3333
Epoch: 2 | Test Loss: 0.55445 | Test Acc: 83.3333
Epoch: 2 | Test Loss: 0.56385 | Test Acc: 79.1667
Epoch: 2 | Test Loss

Analyzing Parameters

In [23]:
for name, param in MarketPredictorModel.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Shape: {param.shape}")
        print(param.data)
        print("-" * 40)

Layer: layer_1.weight | Shape: torch.Size([16, 10])
tensor([[ 0.2855, -0.1026, -0.3004, -0.1612,  0.3087, -0.0544, -0.2885,  0.2669,
          0.2743,  0.2848],
        [ 0.1103, -0.8060,  0.6610,  0.2554, -0.1130, -0.1737,  0.4275, -0.0267,
         -0.1830,  0.1022],
        [ 0.2512,  0.9193, -1.6732,  0.0966,  0.8955, -0.0755, -1.8308,  1.0481,
         -0.2885, -0.3258],
        [ 0.2332,  0.3016,  0.3132, -0.0043,  0.0264, -0.3117,  0.0831,  0.0926,
         -0.0455, -0.2206],
        [ 0.0571,  0.0719,  0.6415, -0.0618,  0.0895,  0.2832,  0.9054, -0.2846,
          0.0477, -0.2134],
        [ 0.2311,  1.6097, -0.3228,  0.0970,  0.7149, -0.1563, -0.1165,  0.7615,
          0.0725,  0.1527],
        [-0.1749,  0.0721, -0.2027, -0.0701, -0.0561, -0.0349,  0.2728,  0.0407,
          0.0602, -0.3040],
        [ 0.3539,  0.5547,  1.1252, -0.1985, -0.0796,  0.1080,  0.8224, -0.4665,
          0.2864,  0.0451],
        [-0.1708,  0.2028, -0.4525,  0.2756, -0.0510, -0.0689, -0.0834, -0.0

In [24]:
timerStamper = pd.read_csv("data/silver/timerStamper1.csv")

correct = 0
total  =0

for market in timerStamper['Market'].tolist():
    try:
        buyScans = pd.read_csv(f'data/bronze/contract_official_buys_{market}.csv')
    except FileNotFoundError:
        print(f"Error: 1")
        continue

    buyScans75Index = max(buyScans['timeStampSinceFirst'].tolist())*.75
    buyScans75 = buyScans[buyScans['timeStampSinceFirst'] < buyScans75Index]


    buysIndex0 = buyScans75[buyScans75['outcomeIndex'] == 0]['investmentAmount']
    buysIndex1 = buyScans75[buyScans75['outcomeIndex'] == 1]['investmentAmount']

    
    prediction = sum(buysIndex0)/(sum(buysIndex0) + sum(buysIndex1))
    prediction = 0 if prediction >= 0.5 else 1
    outcomeIndex = timerStamper[timerStamper['Market'] == market]['OutcomeIndex'].tolist()[0]
    
    if prediction == outcomeIndex:
        correct += 1
    total += 1
    
    
print(f"Correct: {correct} | Total: {total} | Accuracy: {correct/total*100}%")
    
    
    

Correct: 731 | Total: 920 | Accuracy: 79.45652173913044%


In [25]:
trainResults.mean()

AttributeError: 'list' object has no attribute 'mean'

In [ ]:
print("trainData :", trainData.shape)
print("trainResults :", trainResults.shape)

trainData : torch.Size([736, 10])
trainResults : torch.Size([736])
